In [34]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
import plotly.graph_objects as go
from scipy import stats

In [35]:
SCOPES = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', SCOPES)
connection = gspread.authorize(credentials)
wks = connection.open("Example Data").sheet1

#### Generate dataframe from Google Sheets

In [36]:
data = wks.get_all_values()
headers = data.pop(0)

In [37]:
df = pd.DataFrame(data, columns=headers)

In [38]:
df[df.columns] = df[df.columns].apply(pd.to_numeric) 

#### Each row represents stats by day, sorted by :
- x: % of visits w/ high no-show rate patients
- Y: % of visits w/ a no-show status

In [39]:
df

,x,y
0,2,4
1,3,5
2,5,7
3,7,10
4,9,15


#### For each (x, y) point, calculate x squared (x^2) and xy

In [40]:
df['x^2'] = df['x']**2

In [41]:
df['xy'] = df['x'] * df['y']

#### Generate totals row

In [42]:
df_with_total = df.append(df.sum().rename('Total'))

In [43]:
df_with_total

,x,y,x^2,xy
0,2,4,4,8
1,3,5,9,15
2,5,7,25,35
3,7,10,49,70
4,9,15,81,135
Total,26,41,168,263


In [44]:
total = df_with_total.loc[df_with_total.index == 'Total']

In [45]:
number_of_items = len(df)

#### Formula to calculate slope

In [46]:
# m =  N Σ(xy) − Σx*Σy
#      ---------------
#      N Σ(x2) − (Σx)^2

In [47]:
divisor = (number_of_items * total['xy']) - (total['x'] * total['y'])

In [48]:
dividend = (number_of_items * total['x^2']) - (total['x']**2)

In [49]:
m = float(divisor / dividend)

In [50]:
m

1.5182926829268293

#### Formula to calculate intercept

In [51]:
# b =  Σy − m Σx
#      ---------
#          N

In [52]:
divisor = (total['y']) - (m*total['x'])

In [53]:
dividend = number_of_items

In [54]:
b = float(divisor/dividend)

In [55]:
b

0.30487804878048763

#### Assemble the equation of a line for the predicted value

In [56]:
df['predicted_y'] = round((m * df['x']) + b, 2)

In [57]:
predicted_df = df.copy()[['x', 'y', 'predicted_y']]

In [58]:
predicted_df['error'] = predicted_df['predicted_y'] - predicted_df['y']

In [59]:
predicted_df

,x,y,predicted_y,error
0,2,4,3.34,-0.66
1,3,5,4.86,-0.14
2,5,7,7.90,0.90
3,7,10,10.93,0.93
4,9,15,13.97,-1.03


#### If 8% of the visits are w/frequent no-shows, the predicted no-show rate is: 12%

In [68]:
predicted_no_show_rate = round(m * 8 + b, 2)

In [69]:
predicted_no_show_count

12.45

In [62]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['x'].values, y=df['y'].values,
                    mode='lines+markers',
                    name='actual'))
fig.add_trace(go.Scatter(x=predicted_df['x'], y=predicted_df['predicted_y'],
                    mode='lines',
                    name='projected'))
fig.show()

#### Doing the same as above w/SciPy

In [63]:
slope, intercept, r_value, p_value, std_err = stats.linregress(df['x'].values,df['y'].values)

In [64]:
slope

1.518292682926829

In [65]:
intercept

0.30487804878048763

In [66]:
predicted_no_show_count_scipy = round(slope * 8 + intercept, 2)

In [67]:
predicted_no_show_count_scipy

12.45